In [2]:
import multiprocessing
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
from scipy.interpolate import interp1d
import re
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

def tf(x):
    return(np.log10(x))

def decader(x):
    return(x -x%10)

from itertools import groupby
from operator import itemgetter
import pickle as pkl
import re
def patternmaker(x):
    x=np.array(x.notnull())
    x=x.astype(int)
    #print(x)
    val = ''.join(map(str, x))
    #print(val)
    return val

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


def lemmamake(bigram):
    t1=time.time()
    
    #bigram[['modifier','head']] = df.iloc[:,22:26].applymap(binary_values)
    #bigram[['modifier','head']]=lemmamaker(bigram['modifier'].values, bigram['m_pos'].values,bigram['head'].values, bigram['h_pos'].values)
    #=lemmamaker(bigram['head'].values, bigram['h_pos'].values)
    #bigram=bigram.apply(lemmamaker,axis=1)
    
    
    #bigram["modifier"] = bigram["modifier"].map(str) + '_' + bigram["m_pos"]
    #bigram["head"] = bigram["head"].map(str) + '_' + bigram["h_pos"]
    temp_mod=bigram[["modifier","m_pos"]].drop_duplicates()
    t2=time.time()
    print(t2-t1)
    
    #display(temp_mod)
    temp_mod['lemma_mod']=lemmamaker_mod(temp_mod['modifier'].values,temp_mod['m_pos'].values)
    t3=time.time()
    print(t3-t2)
    
    temp_mod["lemma_mod"]=temp_mod["lemma_mod"].map(str) + '_' + temp_mod["m_pos"]
    t4=time.time()
    print(t4-t3)
    
    temp_head=bigram[["head","h_pos"]].drop_duplicates()
    t5=time.time()
    print(t5-t4)
    
    temp_head['lemma_head']=lemmamaker_head(temp_head['head'].values,temp_head['h_pos'].values)
    t6=time.time()
    print(t6-t5)
    
    
    temp_head["lemma_head"]=temp_head["lemma_head"].map(str) + '_' + temp_head["h_pos"]
    t7=time.time()
    print(t7-t6)
    
    
    lemma_replacements={'modifier':dict(zip(temp_mod.modifier.values,temp_mod.lemma_mod.values)),'head':dict(zip(temp_head['head'].values,temp_head.lemma_head.values))}
    t8=time.time()
    print(t8-t7)
    
    
    bigram.replace(lemma_replacements,inplace=True)
    t9=time.time()
    print(t9-t8)
    
    
    #print(type(word))
    #word["modifier"]=lemmatizer.lemmatize(word["modifier"],word.m_pos)+"_"+word.m_pos
    #word["head"]=lemmatizer.lemmatize(word["head"],word.h_pos)+"_"+word.h_pos
    #word.drop(["h_pos","m_pos"],axis=1,inplace=True)
    return bigram


def nayalemma_maker(bigram):
    #t1=time.time()
    bigram['modifier']=lemmamaker_mod(bigram['modifier'].values,bigram['m_pos'].values)
    #t2=time.time()
    #print(t2-t1)
    
    
    bigram["modifier"]=bigram["modifier"].map(str) + '_' + bigram["m_pos"]
    #t3=time.time()
    #print(t3-t2)
    
    bigram['head']=lemmamaker_mod(bigram['head'].values,bigram['h_pos'].values)
    #t4=time.time()
    #print(t4-t3)
    
    bigram["head"]=bigram["head"].map(str) + '_' + bigram["h_pos"]
    #t5=time.time()
    #print(t5-t4)
    return bigram
    
#def lemmamaker(word):
    #word['modifer']=lemmatizer.lemmatize(word["modifier"],word.m_pos)
    #word['head']=lemmatizer.lemmatize(word["head"],word.h_pos)
    #return word

def lemmamaker_mod(mod,mpos):
    
    return list(map(lemmatizer.lemmatize,mod,mpos)) #+"_"+word.m_pos


def lemmamaker_head(hd,hpos):

    return list(map(lemmatizer.lemmatize,hd,hpos)) #+"_"+word.h_pos




def string_operations(bigram):
    
    bigram=bigram.loc[bigram.bigram_pos.str.contains("^.*_(NOUN|VERB|ADJ) .*_(NOUN|VERB|ADJ)$")]

    bigram['modifier_pos'],bigram['head_pos']=bigram['bigram_pos'].str.split().str

    bigram['modifier'], bigram['m_pos'] = bigram['modifier_pos'].str.split('_', 1).str

    bigram['head'], bigram['h_pos'] = bigram['head_pos'].str.split('_', 1).str

    bigram.drop(["bigram_pos","modifier_pos","head_pos"],axis=1,inplace=True)

    return bigram

import time
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
decades=[2000, 1990, 1980, 1970, 1960, 1950, 1940, 1930, 1920, 1900, 1910,
            1890, 1880, 1870, 1850, 1860, 1840, 1830, 1820, 1810, 1800]
replacements={'h_pos':dict(zip(["NOUN","VERB","ADJ"],['n','v','a'])),'m_pos':dict(zip(["NOUN","VERB","ADJ"],['n','v','a']))}

In [3]:
#Generation of bigram files



for letter in bigram_list:
    if len(letter)>2:
        #print(letter)
        bigram_list.remove(letter)
    #elif len(letter.strip("_"))==1:
        #print(letter)
        #bigram_list.remove(letter)
len(bigram_list)

702

In [3]:
def bigram_extracter(letter):
    #t1=time.time()
    #Read Dataset
    path_loc='http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-2gram-20120701-'+letter+'.gz'
    bigram   = pd.read_csv(path_loc, compression='gzip', header=None, sep="\t", quotechar='"',usecols=[0,1,2])

    bigram.columns=['bigram_pos','year','count']
    #t2=time.time()
    #print("End of reading dataset",t2-t1)
    #bigram.info(verbose=False,memory_usage=True)
    
    
    #Converting years to decades
    bigram['year']=decader(bigram.year.values)
    #bigram.drop(["year"],axis=1,inplace=True)
    #t3=time.time()
    #print("End of Converting years to decades",t3-t2)
    #bigram.info(verbose=False,memory_usage=True)
    
        
    #Select decades between 1800 and 2000
    bigram=bigram.loc[bigram.year.isin(decades)]
    #t4=time.time()
    #print("End of Select decades between 1800 and 2000",t4-t3)
    #bigram.info(verbose=False,memory_usage=True)
    
    
    
    #Merging rows based on decades
    bigram=bigram.groupby(['bigram_pos','year'])['count'].sum().to_frame()
    bigram.reset_index(inplace=True)
    #t5=time.time()
    #print("End of Merging rows based on decades",t5-t4) 
    #bigram.info(verbose=False,memory_usage=True)
    
    #all_df=[]
    #cpu_count=30
    #pool=mp.Pool(cpu_count)
    #for temp_df in pool.imap_unordered(string_operations,bigram):
        #all_df.append(temp_df)
    #pool.close()
    #pool.join()
    #display(bigram['bigram_pos'].str.split(" ",1))
    #bigram=bigram.apply(string_operations)
    #bigram['modifier_pos'],bigram['head_pos']=bigram['bigram_pos'].str.split(" ",1).str
    #bigram.drop(["bigram_pos"],axis=1,inplace=True)
    #display(bigram['modifier_pos'].str.split('_', 1))
    #bigram['modifier'], bigram['m_pos'] = bigram['modifier_pos'].str.split('_', 1).str
    #bigram.drop(["modifier_pos"],axis=1,inplace=True)
    #bigram['head'], bigram['h_pos'] = bigram['head_pos'].str.split('_', 1).str
    #bigram.drop(["bigram_pos","modifier_pos","head_pos"],axis=1,inplace=True)
    
    
    #bigram['modifier_pos']=""
    #bigram['head_pos']=""
    #bigram['head']="" 
    #bigram['h_pos']=""
    #bigram['modifier']="" 
    #bigram['m_pos']=""
    #display(bigram)
    bigram=string_operations(bigram)
    #bigram=parallelize_dataframe(bigram,string_operations)
    
    
    #t6=time.time()
    #print("End of separating columns",t6-t5)
    #bigram.info(verbose=False,memory_usage=True)
                
    #Select only Nouns, Verbs and Adjectives
    #bigram=bigram.loc[(bigram.m_pos.isin(["NOUN","VERB","ADJ"])) & (bigram.h_pos.isin(["NOUN","VERB","ADJ"]))]
    #t7=time.time()
    #print("End of selecting only Nouns, Verbs and Adjectives",t7-t6)
    #bigram.info(verbose=False,memory_usage=True)


    
    

    
    #Removing non-english words and words containing non alphabetic characters
    bigram=bigram.loc[(bigram.modifier.str.match('^[a-zA-Z]+$') & (bigram["head"].str.match('^[a-zA-Z]+$')))]
    #t8=time.time()
    #print("End of Removing non-english words and words containing non alphabetic characters",t8-t7)
    #bigram.info(verbose=False,memory_usage=True)

    
    #Reducing all words to lowercase form
    bigram.modifier=bigram.modifier.str.lower()
    bigram["head"]=bigram["head"].str.lower()
    #t9=time.time()
    #print("End of Reducing all words to lowercase form",t9-t8) 
    #bigram.info(verbose=False,memory_usage=True)

    
    #Converting POS tags to ones compliant with the WordNet lemmatizer
    bigram.replace(replacements,inplace=True)
    bigram=bigram.loc[(bigram.m_pos.isin(["n","v","a"])) & (bigram.h_pos.isin(["n","v","a"]))]
    #t10=time.time()
    #print("End of Converting POS tags to ones compliant with the WordNet lemmatizer",t10-t9) 
    #bigram.info(verbose=False,memory_usage=True)

    #return bigram
    #Applying WordNet lemmatizer

    #display(bigram)
    #temp_mod=bigram[["modifier","m_pos"]].drop_duplicates()
    #display(temp_mod)
    #temp_mod['lemma_mod']=temp_mod.apply(lemmamaker_mod,axis=1)
    
    
    #temp_head=bigram[["head","h_pos"]].drop_duplicates()
    #temp_head['lemma_head']=temp_head.apply(lemmamaker_head,axis=1)
    
    
    #lemma_replacements={'modifier':dict(zip(temp_mod.modifier,temp_mod.lemma_mod)),'head':dict(zip(temp_head['head'],temp_head.lemma_head))}

    
    #bigram.replace(lemma_replacements,inplace=True)
    #bigram[['modifier_temp','head_temp']]=bigram.apply(lemmamaker)
    #bigram.drop(["modifer","head"],axis=1,inplace=True)
    
    
    #display(bigram)
    bigram=nayalemma_maker(bigram)
    #t11=time.time()
    #print("End of Applying WordNet lemmatizer",t11-t10) 
    #bigram.info(verbose=False,memory_usage=True)

    
    #Merging words with their pos tags
    #bigram['head']=bigram['head_temp'].astype(str) + '_' + bigram['h_pos'].astype(str)
    #bigram.drop(["head_temp","h_pos"],axis=1,inplace=True)

    #bigram['modifier']=bigram['modifier_temp'].astype(str) + '_' + bigram['m_pos'].astype(str)
    #bigram.drop(["modifier_temp","m_pos","head_temp","h_pos"],axis=1,inplace=True)
    #t12=time.time()
    #print("End of Merging words with their pos tags",t12-t11) 
    #bigram.info(verbose=False,memory_usage=True)

    
    #Merging rows on lemmas
    bigram=bigram.groupby(['modifier','head','year'])['count'].sum().to_frame()
    #bigram=bigram.reset_index(level=['modifier','head','decade'])
    #t12=time.time()
    #print("End of Merging rows on lemmas",t12-t11) 
    #bigram.info(verbose=False,memory_usage=True)

    
    #Selecting words on frequency cutoff of 3
    bigram= bigram.query('count > 9')
    bigram.reset_index(inplace=True)
    #t13=time.time()
    #print("End of Selecting words on frequency cutoff of 3",t13-t12) 
    #bigram.info(verbose=False,memory_usage=True)
    
    #Pivoting the dataset, with the words as rows and the decades as columns
    
    #bigram=pd.pivot_table(bigram,index=['modifier','head'],columns='year',values='count',aggfunc=np.sum)
    #bigram.columns=['1800s','1810s','1820s','1830s','1840s','1850s','1860s','1870s','1880s','1890s','1900s','1910s','1920s','1930s','1940s','1950s','1960s','1970s','1980s','1990s','2000s']
    #display(bigram.info())
    #display(unigram)
    #bigram.reset_index(inplace=True)
    #display(bigram)
    #t14=time.time()
    #print("End of Pivoting the dataset, with the words as rows and the decades as columns",t14-t13)
    #bigram.info(verbose=False,memory_usage=True)

    #outfile_file='/fs/scratch/users/dharpt/bigrams/'+letter+'.csv'
    #with open(outfile_file, 'w') as f:
        #bigram.to_csv(f, sep="\t")
    return(bigram)

In [14]:
#def main():,chunksize=len(unigram_list)
all_df=[]
cpu_count=10
pool=multiprocessing.Pool(cpu_count)
for temp_df in pool.imap_unordered(bigram_extracter,bigram_list):
    all_df.append(temp_df)

pool.join()
pool.close()
#Concatenating all dataframes into 1
bigram = pd.concat(all_df)
#bigram=bigram.groupby(["modifier","head"])['1800s','1810s','1820s','1830s','1840s','1850s','1860s','1870s','1880s','1890s','1900s','1910s','1920s','1930s','1940s','1950s','1960s','1970s','1980s','1990s','2000s'].sum()
bigram=pd.pivot_table(bigram,index=['modifier','head'],columns='year',values='count',aggfunc=np.sum)
bigram.columns=['1800s','1810s','1820s','1830s','1840s','1850s','1860s','1870s','1880s','1890s','1900s','1910s','1920s','1930s','1940s','1950s','1960s','1970s','1980s','1990s','2000s']
bigram['cf']=np.log10(bigram.sum(axis=1))
bigram['presence']=bigram.drop('cf',axis=1).count(axis=1)


#unigram['presence']=unigram.count(axis=1)
#unigram['idf']=np.log10(18/unigram['presence'])
bigram['presence']=bigram["presence"].astype('category')
bigram[bigram.columns[:-2]]=bigram[bigram.columns[:-2]].apply(tf, axis=1)
bigram['pattern']=bigram[bigram.columns[:-2]].apply(patternmaker,axis=1)
display(bigram.head(10))
bigram.tail(10)

,modifier,head,year,count
0,a_a,a_a,1930.000,12
1,a_a,a_a,1940.000,14
2,a_a,a_a,1950.000,18
3,a_a,a_a,1960.000,16
4,a_a,a_a,1970.000,11
5,a_a,a_a,1980.000,43
6,a_a,a_a,1990.000,100
7,a_a,a_a,2000.000,74
8,a_a,a_n,1910.000,11
9,a_a,a_n,1930.000,17


Process ForkPoolWorker-3:
KeyboardInterrupt
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-7:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/m

In [5]:
#with open('/fs/scratch/users/dharpt/bigrams/bigrams.csv', 'w') as f:
    #bigram.to_csv(f, sep="\t")

In [4]:
bigram=pd.read_csv("",sep="\t",index_col=[0,1])
bigram

FileNotFoundError: File b'bigrams/bigrams.csv' does not exist

In [ ]:
#with open('/fs/scratch/users/dharpt/bigrams/bigrams.csv', 'w') as f:
    #bigram.to_csv(f, sep="\t")